# ClassicModels → Conception d'une base **document** (JSON/MongoDB)

**Livrables dans ce notebook :**
1) Le **nombre de collections** et leur rôle
2) Le **schéma type** des documents dans chaque collection
3) Le **pseudo‑algorithme** de passage SQL → document



## 1) Nombre de collections et rôle

:
- **Mapping ** (proche du SQL) : `customers`, `employees`, `offices`, `orders`, `orderdetails`, `payments`, `products`.
.

**À faire :** Décrivez ici vos collections et leur rôle.


In [22]:
#Mapping 1:1 (proche du SQL)
#Quand choisir ça ?

#Tu veux une migration simple et fidèle au relationnel.

#Tes requêtes accepteront plusieurs $lookup (jointures) pour reconstruire une commande.

In [5]:
collections_design = """
- customers   : clients (coordonnées, pays, commercial rattaché via salesRepEmployeeNumber)
- employees   : employés (commerciaux) avec officeCode
- offices     : bureaux (adresse, ville, pays)
- products    : catalogue produits (productLine, buyPrice, etc.)
- orders      : commandes (dates, status, customerNumber)
- orderdetails: lignes de commande (productCode, quantityOrdered, priceEach, orderLineNumber)
- payments    : paiements des clients (paymentDate, amount, checkNumber)
"""
print(collections_design)




- customers   : clients (coordonnées, pays, commercial rattaché via salesRepEmployeeNumber)
- employees   : employés (commerciaux) avec officeCode
- offices     : bureaux (adresse, ville, pays)
- products    : catalogue produits (productLine, buyPrice, etc.)
- orders      : commandes (dates, status, customerNumber)
- orderdetails: lignes de commande (productCode, quantityOrdered, priceEach, orderLineNumber)
- payments    : paiements des clients (paymentDate, amount, checkNumber)



## 2) Schéma type des documents (JSON) par collection

Recommandation : utiliser des **_id** déterministes (stables).

Exemples :

**1:1**
`customers` ⇒ `{ "_id": 103, "customerNumber": 103, "customerName": "Atelier graphique", ... }`

`orderdetails` ⇒ `{ "_id": "10100#S10_1678#1", "orderNumber": 10100, "productCode": "S10_1678", "orderLineNumber": 1, ... }`


 **À faire :** Décrivez les champs attendus (nom, type, clé) pour chaque collection.


In [8]:
document_schemas = {
    # Documents de visionnage dénormalisés regroupant client, film et location
    "visionnage" : {
        "_id" : "rental_id", #ID (cle primaire) de la table rental{rental_id}
        "fields" : {
            # "type": "object", Informations du client
            "customer_info" : {         
                "customer_id" : "int",     # ID de la table customer
                "first_name" : "string",   # Prénom du client
                "last_name" : "string"     # Nom du client"
            },
            # "type": "object", Informations du film avec ses genres
            "film_info" : {
                "film_id" : "int",         # ID de la table film
                "title" : "string",        # Titre du film
                "release_year" : "int",    # Année de sortie
                "genres": ["array"]        # Liste des genres du film     
            },
            # "type": "object", Informations de location
            "rental_info" : {
                "rental_date" : "ISODate", # Date de début de location
                "return_date" : "ISODate", # Date de retour
            }
        }
    }
}


document_schemas

{'visionnage': {'_id': 'rental_id',
  'fields': {'customer_info': {'customer_id': 'int',
    'first_name': 'string',
    'last_name': 'string'},
   'film_info': {'film_id': 'int',
    'title': 'string',
    'release_year': 'int',
    'genres': ['array']},
   'rental_info': {'rental_date': 'ISODate', 'return_date': 'ISODate'}}}}

### Exemples de documents complets

In [10]:
visionnage_exemples = [
    # Document 1 - Visionnage complet avec retour
    {
        "_id" : "rental_76",
        "customer_info" : {
            "customer_id" : 130,
            "first_name" : "SUSAN",
            "last_name" : "WILSON"
        },
        "film_info" : {
            "film_id" : 535,
            "title" : "LOVE SUICIDES",
            "release_year" : 2006,
            "genres" : ["Drama", "Romance"]
        },
        "rental_info": {
            "rental_date" : "2005-07-31 07:18:56",
            "return_date" : "2005-08-08 03:18:56"
        }
    },
    
    # Document 2 - Visionnage Multi-genres
    {
        "_id": "rental_123",
        "customer_info": {
            "customer_id": 45,
            "first_name": "JOHN",
            "last_name": "DOE"
        },
        "film_info": {
            "film_id": 789,
            "title": "MATRIX REVOLUTIONS",
            "release_year": 2003,
            "genres": ["Action", "Sci-Fi", "Thriller"]
        },
        "rental_info": {
            "rental_date": "2005-08-15 14:22:10",
            "return_date": "2005-08-20 11:30:45"
        }
    },
    
    # Document 3 - Visionnage en cours (non retourné)
    {
        "_id": "rental_245",
        "customer_info": {
            "customer_id": 289,
            "first_name": "MARIE",
            "last_name": "DUPONT"
        },
        "film_info": {
            "film_id": 642,
            "title": "INCEPTION",
            "release_year": 2010,
            "genres": ["Sci-Fi", "Action", "Mystery"]
        },
        "rental_info": {
            "rental_date": "2005-08-25 09:15:30",
            "return_date": None
        }
    }
]
visionnage_exemples

[{'_id': 'rental_76',
  'customer_info': {'customer_id': 130,
   'first_name': 'SUSAN',
   'last_name': 'WILSON'},
  'film_info': {'film_id': 535,
   'title': 'LOVE SUICIDES',
   'release_year': 2006,
   'genres': ['Drama', 'Romance']},
  'rental_info': {'rental_date': '2005-07-31T07:18:56',
   'return_date': '2005-08-08T03:18:56'}},
 {'_id': 'rental_123',
  'customer_info': {'customer_id': 45,
   'first_name': 'JOHN',
   'last_name': 'DOE'},
  'film_info': {'film_id': 789,
   'title': 'MATRIX REVOLUTIONS',
   'release_year': 2003,
   'genres': ['Action', 'Sci-Fi', 'Thriller']},
  'rental_info': {'rental_date': '2005-08-15T14:22:10',
   'return_date': '2005-08-20T11:30:45'}},
 {'_id': 'rental_245',
  'customer_info': {'customer_id': 289,
   'first_name': 'MARIE',
   'last_name': 'DUPONT'},
  'film_info': {'film_id': 642,
   'title': 'INCEPTION',
   'release_year': 2010,
   'genres': ['Sci-Fi', 'Action', 'Mystery']},
  'rental_info': {'rental_date': '2005-08-25T09:15:30', 'return_date':

## 3) Pseudo‑algorithme SQL → document

Expliquez **comment** transformer les tables relationnelles en documents JSON selon votre choix.

Exemple **1:1** :
- Lire chaque table, définir `_id`, exporter/insérer, créer les index.



 **À faire :** Rédigez votre pseudo‑algorithme ci‑dessous.


In [9]:
pseudo_algo = """
Objectif : Migrer ClassicModels (SQL) vers un modèle document 1:1 (une table -> une collection).

0) Préparation
   - Lister les tables: Customers, Employees, Offices, Orders, OrderDetails, Payments, Products.
   - Choisir des _id déterministes (stables) pour chaque collection :
     * customers._id = customerNumber
     * employees._id = employeeNumber
     * offices._id   = officeCode
     * orders._id    = orderNumber
     * products._id  = productCode
     * orderdetails._id = "{orderNumber}#{productCode}#{orderLineNumber}"
     * payments._id     = "{customerNumber}#{checkNumber}"

1) Lecture SQL -> DataFrames
   Pour chaque table T :
     - df_T = SELECT * FROM T;
     - Vérifier le nombre de lignes et les colonnes attendues.

2) Construction des documents JSON (1:1)
   - Pour T dans {Customers, Employees, Offices, Orders, Products} :
       * Ajouter une colonne _id = clé naturelle correspondante.
       * (Option) Normaliser les types simples : int, number (float), string; convertir dates en ISO "YYYY-MM-DD".
   - Pour OrderDetails :
       * Ajouter _id = f"{orderNumber}#{productCode}#{orderLineNumber}".
   - Pour Payments :
       * Ajouter _id = f"{customerNumber}#{checkNumber}".

3) Export ou insertion
   OPTION A — Export fichiers JSON Lines (.jsonl), un fichier par collection :
       * Écrire chaque document en une ligne JSON.
   OPTION B — Insertion MongoDB (pymongo) :
       * db = MongoClient(...).classicmodels_flat
       * db.<collection>.delete_many({}) (optionnel pour repartir propre)
       * db.<collection>.insert_many(documents)

4) Index / contraintes utiles
   - db.orders.create_index("customerNumber")
   - db.orderdetails.create_index([("orderNumber",1),("productCode",1),("orderLineNumber",1)], unique=True)
   - db.payments.create_index([("customerNumber",1),("checkNumber",1)], unique=True)
   - db.products.create_index("productLine")
   - (Option) index uniques sur les clés naturelles si _id différent (ici _id couvre déjà).

5) Vérifications fonctionnelles (post-migration)
   - Q1 : clients sans commande (lookup customers -> orders ; filtrer size==0)
   - Totaux commandes/pays, nb de commandes par produit, etc.
   - Contrôler que les cardinalités (nb docs par collection) == nb lignes SQL d’origine.

6) Qualité des données & reprises
   - Gérer NULL : convertir en null JSON (ou valeurs par défaut) au lieu de chaînes "NULL".
   - Vérifier doublons sur les clés naturelles avant insertion (pour éviter 11000 DuplicateKeyError).
   - Journaliser les erreurs d’insertion, corriger et rejouer si besoin.

7) Livraison
   - Livrer : fichiers .jsonl (si export) et/ou script notebook d’insertion.
   - Documenter : mapping table -> collection, choix des _id, index, et jeux de requêtes de validation.
"""
print(pseudo_algo)




Objectif : Migrer ClassicModels (SQL) vers un modèle document 1:1 (une table -> une collection).

0) Préparation
   - Lister les tables: Customers, Employees, Offices, Orders, OrderDetails, Payments, Products.
   - Choisir des _id déterministes (stables) pour chaque collection :
     * customers._id = customerNumber
     * employees._id = employeeNumber
     * offices._id   = officeCode
     * orders._id    = orderNumber
     * products._id  = productCode
     * orderdetails._id = "{orderNumber}#{productCode}#{orderLineNumber}"
     * payments._id     = "{customerNumber}#{checkNumber}"

1) Lecture SQL -> DataFrames
   Pour chaque table T :
     - df_T = SELECT * FROM T;
     - Vérifier le nombre de lignes et les colonnes attendues.

2) Construction des documents JSON (1:1)
   - Pour T dans {Customers, Employees, Offices, Orders, Products} :
       * Ajouter une colonne _id = clé naturelle correspondante.
       * (Option) Normaliser les types simples : int, number (float), string; con

## 1) Lecture SQL → DataFrames + _id stables

### MongoDB

In [88]:
import sqlite3
import pandas as pd
import json
from datetime import datetime

# Adapte le chemin si besoin
# conn = sqlite3.connect('sakila_master.db')
conn = sqlite3.connect('sqlite-sakila.db')

def sql_df(q: str) -> pd.DataFrame:
    return pd.read_sql_query(q, conn)

# Création des dossiers d'export
EXPORTS_JSON = Path("mongodb_exports")
EXPORTS_CSV = Path("neo4j_exports")
EXPORTS_JSON.mkdir(exist_ok=True)
EXPORTS_CSV.mkdir(exist_ok=True)

# # Chargement
# df_rental         = sql_df("SELECT * FROM rental")
df_staff          = sql_df("SELECT * FROM staff")
df_store          = sql_df("SELECT * FROM store")
# df_customer       = sql_df("SELECT * FROM customer")
# df_inventory      = sql_df("SELECT * FROM inventory")
# df_film           = sql_df("SELECT * FROM film")
# df_film_category  = sql_df("SELECT * FROM film_category")
# df_category       = sql_df("SELECT * FROM category")
display(df_staff)
df_store
# # _id déterministes
# df_rental["_id"]         = df_rental["rental_id"]
# df_staff["_id"]          = df_staff["staff_id"]
# df_store["_id"]          = df_store["store_id"]
# df_customer["_id"]       = df_customer["customer_id"]
# df_inventory["_id"]      = df_inventory["inventory_id"]
# df_film["_id"]           = df_film["film_id"]
# df_category["_id"]       = df_category["category_id"]
# df_film_category["_id"]  = df_film_category.apply(lambda r: f"{r['film_id']}#{r['category_id']}", axis=1)

# {   "rental":len(df_rental), 
#     "staff":len(df_staff), 
#     "store":len(df_store), 
#     "customer":len(df_customer), 
#     "inventory":len(df_inventory), 
#     "film":len(df_film),
#     "film_category":len(df_film_category),
#     "category":len(df_category)}

,staff_id,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,None,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2021-03-06 15:52:00
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,8cb2237d0679ca88db6464eac60da96345513964,2021-03-06 15:52:00


,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2021-03-06 15:52:00
1,2,2,2,2021-03-06 15:52:00


### Neo4j

In [90]:
# Données pour les nœuds Customer
df_customers_neo = sql_df("""
    SELECT 
        customer_id as 'customerId:ID(Customer)',
        first_name as firstName,
        last_name as lastName
    FROM customer
""")

# Données pour les nœuds Film
df_films_neo = sql_df("""
    SELECT 
        film_id as 'filmId:ID(Film)',
        title,
        release_year as releaseYear
    FROM film
""")

# Données pour les nœuds Category
df_categories_neo = sql_df("""
    SELECT 
        category_id as 'categoryId:ID(Category)',
        name as categoryName
    FROM category
""")

# Données pour les nœuds Staff (hiérarchie)
df_staff_neo = sql_df("""
    SELECT 
        staff_id as 'staffId:ID(Staff)',
        first_name as firstName,
        last_name as lastName
    FROM staff
""")

# Relations: Customer -[WATCHED]-> Film
df_watched_relations = sql_df("""
    SELECT 
        c.customer_id as ':START_ID(Customer)',
        f.film_id as ':END_ID(Film)',
        r.rental_date as rentalDate,
        'WATCHED' as ':TYPE'
    FROM rental r
    JOIN customer c ON r.customer_id = c.customer_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
""")

# Relations: Film -[IN_CATEGORY]-> Category
df_film_category_relations = sql_df("""
    SELECT 
        fc.film_id as ':START_ID(Film)',
        fc.category_id as ':END_ID(Category)',
        'IN_CATEGORY' as ':TYPE'
    FROM film_category fc
""")

# Relations: Staff hierarchy 
# df_staff_hierarchy = sql_df("""
#     SELECT 
#         s1.staff_id as `:START_ID(Staff)`,
#         s2.staff_id as `:END_ID(Staff)`,
#         'REPORTS_TO' as `:TYPE`
#     FROM staff s1
#     JOIN store st ON s1.staff_id = st.manager_staff_id
#     JOIN staff s2 ON s2.store_id = st.store_id AND s2.staff_id != s1.staff_id
# """)
# SOLUTION FINALE POUR VOTRE PROJET
# Relations: Staff -[MANAGES]-> Store
# Aucune relation hiérarchique entre staff
df_staff_hierarchy = pd.DataFrame(columns=[':START_ID(Staff)', ':END_ID(Staff)', ':TYPE'])

print("ℹ️  Information: Aucune relation hiérarchique entre le staff dans la base Sakila")
print("Chaque manager gère son store indépendamment")

print(df_staff_hierarchy)
df_staff_hierarchy

ℹ️  Information: Aucune relation hiérarchique entre le staff dans la base Sakila
Chaque manager gère son store indépendamment
Empty DataFrame
Columns: [:START_ID(Staff), :END_ID(Staff), :TYPE]
Index: []


,:START_ID(Staff),:END_ID(Staff),:TYPE


## 2) Export data

### JSON LINE

In [91]:
def create_denormalized_rentals():    
    query = """
    SELECT 
        -- Rental
        r.rental_id,
        r.rental_date,
        r.return_date,
        -- Customer
        c.customer_id,
        c.first_name as customer_first_name,
        c.last_name as customer_last_name,
        -- Film 
        f.film_id,
        f.title as film_title,
        f.release_year as film_year,
        -- Genre
        GROUP_CONCAT(DISTINCT cat.name) as categories
    FROM rental r
    JOIN customer c ON r.customer_id = c.customer_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    LEFT JOIN film_category fc ON f.film_id = fc.film_id
    LEFT JOIN category cat ON fc.category_id = cat.category_id
    GROUP BY r.rental_id
    --LIMIT 500  -- Pour le test, ajustez selon vos besoins
    """
    
    df_rentals = sql_df(query)

    # Création des documents MongoDB
    documents = []
    for _, row in df_rentals.iterrows():
        doc = {
            "_id": row['rental_id'],
            "customer_info": {
                "customer_id": row['customer_id'],
                "first_name": row['customer_first_name'],
                "last_name": row['customer_last_name']
            },
            "film_info": {
                "film_id": row['film_id'],
                "title": row['film_title'],
                "release_year": row['film_year'],
                "genres": row['categories'].split(',') if row['categories'] else []
            },
            "rental_info": {
                "rental_date": row['rental_date'],
                "return_date": row['return_date']
            }
        }
        documents.append(doc)
    
    return pd.DataFrame(documents)

In [92]:
# Création des dossiers d'export
EXPORTS_JSON = Path("mongodb_exports")
EXPORTS_CSV = Path("neo4j_exports")
EXPORTS_JSON.mkdir(exist_ok=True)
EXPORTS_CSV.mkdir(exist_ok=True)

def to_jsonl(df: pd.DataFrame, out: Path):
    with open(out, "w", encoding="utf-8") as f:
        for rec in df.to_dict(orient="records"):
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    return out

def to_csv(df: pd.DataFrame, out: Path):
    df.to_csv(out, index=False, encoding='utf-8')
    return out

### CSV

In [95]:
# Export des fichiers CSV pour Neo4j
neo4j_exports = {
    # Nœuds
    "customers": to_csv(df_customers_neo, EXPORTS_CSV/"customers.csv"),
    "films": to_csv(df_films_neo, EXPORTS_CSV/"films.csv"),
    "categories": to_csv(df_categories_neo, EXPORTS_CSV/"categories.csv"),
    "staff": to_csv(df_staff_neo, EXPORTS_CSV/"staff.csv"),
    
    # Relations
    "watched_relations": to_csv(df_watched_relations, EXPORTS_CSV/"watched_relations.csv"),
    "film_category_relations": to_csv(df_film_category_relations, EXPORTS_CSV/"film_category_relations.csv"),
    # "staff_hierarchy": to_csv(df_staff_hierarchy, EXPORTS_CSV/"staff_hierarchy.csv"),
}

In [103]:
# Création des données denormalisées
df_rentals_denorm = create_denormalized_rentals()
print(f"📊 Documents rentals créés: {len(df_rentals_denorm)}")

print("=== EXPORT NEO4J - CSV ===")
mongodb_exp = {
    "rentals":   to_jsonl(df_rentals_denorm, EXPORTS_JSON/"visionnage.jsonl")
}

📊 Documents rentals créés: 16044
=== EXPORT NEO4J - CSV ===


## Scripts Neo4j

In [97]:
# Génération du script Cypher d'import
cypher_script = """
-- SCRIPT CYPHER POUR IMPORT NEO4J
-- Étape 3: Migration des données Neo4j

-- 1. Création des contraintes
CREATE CONSTRAINT customer_id IF NOT EXISTS FOR (c:Customer) REQUIRE c.customerId IS UNIQUE;
CREATE CONSTRAINT film_id IF NOT EXISTS FOR (f:Film) REQUIRE f.filmId IS UNIQUE;
CREATE CONSTRAINT category_id IF NOT EXISTS FOR (cat:Category) REQUIRE cat.categoryId IS UNIQUE;
CREATE CONSTRAINT staff_id IF NOT EXISTS FOR (s:Staff) REQUIRE s.staffId IS UNIQUE;

-- 2. Import des nœuds Customer
LOAD CSV WITH HEADERS FROM 'file:///customers.csv' AS row
CREATE (c:Customer {
    customerId: toInteger(row.customerId),
    firstName: row.firstName,
    lastName: row.lastName,
    email: row.email,
    joinDate: datetime(row.joinDate)
});

-- 3. Import des nœuds Film
LOAD CSV WITH HEADERS FROM 'file:///films.csv' AS row
CREATE (f:Film {
    filmId: toInteger(row.filmId),
    title: row.title,
    releaseYear: toInteger(row.releaseYear),
    rating: row.rating,
    rentalRate: toFloat(row.rentalRate)
});

-- 4. Import des nœuds Category
LOAD CSV WITH HEADERS FROM 'file:///categories.csv' AS row
CREATE (cat:Category {
    categoryId: toInteger(row.categoryId),
    categoryName: row.categoryName
});

-- 5. Import des nœuds Staff
LOAD CSV WITH HEADERS FROM 'file:///staff.csv' AS row
CREATE (s:Staff {
    staffId: toInteger(row.staffId),
    firstName: row.firstName,
    lastName: row.lastName,
    email: row.email,
    storeId: toInteger(row.storeId)
});

-- 6. Création des relations WATCHED
LOAD CSV WITH HEADERS FROM 'file:///watched_relations.csv' AS row
MATCH (c:Customer {customerId: toInteger(row[":START_ID(Customer)"])})
MATCH (f:Film {filmId: toInteger(row[":END_ID(Film)"])})
CREATE (c)-[:WATCHED {rentalDate: datetime(row.rentalDate)}]->(f);

-- 7. Création des relations IN_CATEGORY
LOAD CSV WITH HEADERS FROM 'file:///film_category_relations.csv' AS row
MATCH (f:Film {filmId: toInteger(row[":START_ID(Film)"])})
MATCH (cat:Category {categoryId: toInteger(row[":END_ID(Category)"])})
CREATE (f)-[:IN_CATEGORY]->(cat);

-- 8. Création des relations REPORTS_TO
LOAD CSV WITH HEADERS FROM 'file:///staff_hierarchy.csv' AS row
MATCH (s1:Staff {staffId: toInteger(row[":START_ID(Staff)"])})
MATCH (s2:Staff {staffId: toInteger(row[":END_ID(Staff)"])})
CREATE (s1)-[:REPORTS_TO]->(s2);

-- 9. Validation des imports
MATCH (n) RETURN labels(n) as label, count(*) as count
ORDER BY label;
"""

# Sauvegarde du script Cypher
with open(EXPORTS_CSV/"import_neo4j.cypher", "w", encoding="utf-8") as f:
    f.write(cypher_script)

## 4) Index utiles (si tu as inséré dans Mongo)

In [98]:
# db["orders"].create_index("customerNumber")
# db["orderdetails"].create_index([("orderNumber",1),("productCode",1),("orderLineNumber",1)], unique=True)
# db["payments"].create_index([("customerNumber",1),("checkNumber",1)], unique=True)
# db["products"].create_index("productLine")
# print("Index créés.")


## 5) Vérification rapide (ex. Q1 clients sans commande, côté Mongo)

In [99]:
# pipeline_q1 = [
#   {"$lookup": {"from":"orders", "localField":"customerNumber", "foreignField":"customerNumber", "as":"orders"}},
#   {"$match":  {"$expr": {"$eq":[{"$size":"$orders"}, 0]}}},
#   {"$project":{"_id":0,"customerNumber":1,"customerName":1}},
#   {"$sort":   {"customerNumber":1}}
# ]
# list(db["customers"].aggregate(pipeline_q1))[:10]


In [100]:
# df_staff